In [3]:
import pandas as pd 
from bs4 import BeautifulSoup
import re

In [2]:
path_retrieved_html_success = '../../data/retrieved_html_success.csv'
df_retrieved_html_success = pd.read_csv(path_retrieved_html_success)

## Performing dataset cleaning with bs4

In [3]:
def extract_pure_html(html):
    """
    Isola apenas o conteúdo dentro da tag <html>...</html>
    """
    match = re.search(r'<html.*?>.*?</html>', html, flags=re.DOTALL | re.IGNORECASE)
    return match.group(0) if match else ''

In [4]:
def remove_js_css(html):
    """
    Remove os scripts e estilos do HTML.
    """
    soup = BeautifulSoup(html, 'html.parser')
    for tag in soup(['script', 'style']):
        tag.decompose()
    return str(soup)

In [5]:
def extract_text(html):
    """
    Remove todas as tags HTML e extrai somente o texto visível.
    """
    html_sem_js_css = remove_js_css(html)
    html_limpo = extract_pure_html(html_sem_js_css)
    soup = BeautifulSoup(html_limpo, 'html.parser')
    return soup.get_text(separator=' ', strip=True)

In [6]:
df_retrieved_html_success_clean = df_retrieved_html_success.copy()
df_retrieved_html_success_clean['html_content'] = df_retrieved_html_success_clean['html_content'].apply(extract_text)

In [7]:
df_retrieved_html_success_clean.to_csv('../../data/retrieved_text_success.csv', index=False)

## Extracting the first html to do the cleaning manually

In [6]:
first_html = df_retrieved_html_success.iloc[0]['html_content']

In [ ]:
with open('../data/html_dirty.txt', 'w') as f:
    f.write(first_html)

In [ ]:
html_clean_bs4 = extract_text(first_html)
with open('../data/html_clean_bs4.txt', 'w') as f:
    f.write(html_clean_bs4)

# Concatenating the text_sucess dataset with the summaries.

In [4]:
path_text_success = '../../data/text_success.csv'
path_raw_data = '../../data/raw_data.csv'

In [15]:
df_text_success = pd.read_csv(path_text_success)
df_raw_data = pd.read_csv(path_raw_data)

In [16]:
df_text_success.head()

,expanded_url,short_url,html_content
0,https://midiamax.uol.com.br/cotidiano/economia...,https://bit.ly/3sFuhrg,"Em alerta para estiagem, associação inicia pre..."
1,https://www.campogrande.ms.gov.br/cgnoticias/n...,https://www.campogrande.ms.gov.br/cgnoticias/n...,Campanha de vacinação antirrábica começa nesta...
2,https://www.campograndenews.com.br/cidades/cap...,https://bit.ly/3lUcz1t,Queimada atinge casa de festas e empresário es...
3,https://www.ms.gov.br/com-safra-do-milho-ating...,https://bit.ly/3g4FlsE,Portal Ãnico ms.gov.br You need to enable Jav...
4,https://midiamax.uol.com.br/cotidiano/2021/fre...,https://bit.ly/3sjQ0UY,Frente parlamentar discute formas de combate c...


In [17]:
df_raw_data.head()

,short_url,title,summary
0,https://bit.ly/3sFuhrg,"Em alerta para estiagem, associação inicia pre...",A estiagem prolongada em 2021 fez com diretore...
1,https://bit.ly/3lUcz1t,Queimada atinge casa de festas e empresário es...,NaN
2,https://bit.ly/3g4FlsE,Com safra do milho atingida por geada e estiag...,NaN
3,https://bit.ly/3g7mn4w,Polícia tem suspeito de estuprar jovem que pre...,NaN
4,https://bit.ly/3sjQ0UY,Frente parlamentar discute formas de combate c...,NaN


In [18]:
df_output = df_text_success.copy()

In [22]:
df_aux = pd.DataFrame(columns=["summary"])

for _, row in df_text_success.iterrows():
    match = df_raw_data[df_raw_data['short_url'] == row['short_url']]
    if not match.empty:
        summary_value = match.iloc[0]['summary']
    else:
        summary_value = pd.NA
    df_aux = pd.concat([df_aux, pd.DataFrame({"summary": [summary_value]})], ignore_index=True)

df_output['summary'] = df_aux['summary']
    

In [24]:
df_output.head(-1)

,expanded_url,short_url,html_content,summary
0,https://midiamax.uol.com.br/cotidiano/economia...,https://bit.ly/3sFuhrg,"Em alerta para estiagem, associação inicia pre...",A estiagem prolongada em 2021 fez com diretore...
1,https://www.campogrande.ms.gov.br/cgnoticias/n...,https://www.campogrande.ms.gov.br/cgnoticias/n...,Campanha de vacinação antirrábica começa nesta...,NaN
2,https://www.campograndenews.com.br/cidades/cap...,https://bit.ly/3lUcz1t,Queimada atinge casa de festas e empresário es...,NaN
3,https://www.ms.gov.br/com-safra-do-milho-ating...,https://bit.ly/3g4FlsE,Portal Ãnico ms.gov.br You need to enable Jav...,NaN
4,https://midiamax.uol.com.br/cotidiano/2021/fre...,https://bit.ly/3sjQ0UY,Frente parlamentar discute formas de combate c...,NaN
...,...,...,...,...
6584,https://goias.gov.br/saude-investiga-e-acompan...,https://tinyurl.com/3626tuaf,Saúde investiga e acompanha surto de DDA em mu...,A Secretaria de Estado da Saúde de Goiás (SES-...
6585,https://vnexplorer.net/chile-autoridades-sanit...,https://bit.ly/3C1XBes,Verified News Explorer Network US AU CA AE PH ...,A Secretaria Regional de Saúde de La Araucanía...
6586,https://timbancavirtual.com.br/maislidas/23018...,https://bit.ly/3wzeCvt,Página não encontrada – Banca Virtual Ir para ...,Cinquenta e três casos da hepatite aguda miste...
6587,https://www.correiofrancisquense.com.br/notici...,https://bit.ly/3efp1HB,Correio Francisquense lança parceria inédita c...,"A região Sul do Brasil, um dos principais dest..."


In [25]:
df_output.to_csv('../../data/text_success_with_summary.csv', index=False)

In [4]:
df_text_success_with_summary = pd.read_csv('../../data/text_success_with_summary.csv')


In [5]:
df_sample = df_text_success_with_summary.sample(n=20, random_state=1)
df_sample.to_csv('../../data/text_success_with_summary_sample.csv', index=False)